In [1]:
import plotly.io as pio
import plotly.express as px
import plotly.offline as py
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
import ipywidgets as widgets

# 1. Load dữ liệu

In [3]:
df = pd.read_csv('data/supermarket_sales_vn.csv')
df.head()

,Invoice ID,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,Rating
0,750-67-8428,Hà Nội,Member,Nữ,Health and beauty,74.69,7,26.1415,548.9715,1/5/19,13:08,Ewallet,522.83,9.1
1,226-31-3081,TP HCM,Normal,Nữ,Electronic accessories,15.28,5,3.8200,80.2200,3/8/19,10:29,Cash,76.40,9.6
2,631-41-3108,Hà Nội,Normal,Nam,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/19,13:23,Credit card,324.31,7.4
3,123-19-1176,Hà Nội,Member,Nam,Health and beauty,58.22,8,23.2880,489.0480,1/27/19,20:33,Ewallet,465.76,8.4
4,373-73-7910,Hà Nội,Normal,Nam,Sports and travel,86.31,7,30.2085,634.3785,2/8/19,10:37,Ewallet,604.17,5.3


In [4]:
df_CN_counts = df['City'].value_counts()
df_CN_counts

TP HCM     490
Hà Nội     274
Đà Nẵng    236
Name: City, dtype: int64

In [5]:
df['Product line'].value_counts()

Fashion accessories       178
Food and beverages        174
Electronic accessories    170
Sports and travel         166
Home and lifestyle        160
Health and beauty         152
Name: Product line, dtype: int64

# 2. So sánh theo tổng số đơn hàng

In [13]:
fig = go.Figure(layout=go.Layout(width=400, height=400, 
                                 title="So sánh tổng đơn theo Chi Nhánh"))
goBar = go.Bar(x=df_CN_counts.index, y=df_CN_counts.values, 
               marker=dict(color=['red','green','blue']))
#goScatter = go.Pie(labels=df_CN_counts.index, values=df_CN_counts.values)
fig.add_trace(goBar)
#fig.add_trace(goScatter)

In [12]:
# Tham khảo subplots: https://plotly.com/python/subplots/
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, 
                    subplot_titles=("So Sánh Theo Tổng Số", "So Sánh Theo Tỷ Lệ"), 
                    column_widths=[0.6, 0.4],
                    specs=[[{"type": "xy"}, {"type": "domain"}]]
                   )
goBar = go.Bar(x=df_CN_counts.index, y=df_CN_counts.values, 
               width=0.5,
               marker=dict(color=['red','green','blue']),
               showlegend=False
              )
goPie = go.Pie(labels=df_CN_counts.index, values=df_CN_counts.values, 
               marker=dict(colors=['red','green','blue']))
fig.add_trace(goBar, row=1, col=1)
fig.add_trace(goPie, row=1, col=2)


**Tạo interactive figure so sánh tổng số đơn theo các cột dữ liệu**

In [14]:
@widgets.interact(thongso=['City', 'Gender', 'Product line', 'Payment'])
def SoSanh1(thongso):
    df_counts = df[thongso].value_counts()
    #print(df_counts)
    fig = make_subplots(rows=1, cols=2, 
                        subplot_titles=("So Sánh Theo Tổng Số", "So Sánh Theo Tỷ Lệ"), 
                        column_widths=[0.6, 0.4],
                        specs=[[{"type": "xy"}, {"type": "domain"}]]
                       )
    goBar = go.Bar(x=df_counts.index, y=df_counts.values, 
                   width=0.5,
                   #marker=dict(color=['red','green','blue']),
                   showlegend=False
                  )
    goPie = go.Pie(labels=df_counts.index, values=df_counts.values,
                   #textinfo='label+percent',
                   showlegend=True,
                   hole=.3,
                   #marker=dict(colors=['red','green','blue'])
                  )
    fig.add_trace(goBar, row=1, col=1)
    fig.add_trace(goPie, row=1, col=2)
    fig.update_layout(width=800, height=400, title=f'BIỂU ĐỒ SO SÁNH: {thongso}')
    fig.show()
#SoSanh1('Gender')

interactive(children=(Dropdown(description='thongso', options=('City', 'Gender', 'Product line', 'Payment'), v…

**Hiển thị so sánh tổng giá trị đơn (doanh thu) theo địa điểm**

In [15]:
tongCN = df.groupby('City')['Total'].sum()
tongCN = tongCN.reset_index()
tongCN

,City,Total
0,Hà Nội,85584.3555
1,TP HCM,161644.7490
2,Đà Nẵng,75737.6445


In [16]:
toaDoDiaDiem = pd.DataFrame({'City': ['Hà Nội', 'Đà Nẵng', 'TP HCM'], 
                    'lat': [21.0294498, 16.047079, 10.762622],
                    'lon': [105.8544441, 108.206230, 106.660172]}
                  )
tongCN = pd.merge(tongCN, toaDoDiaDiem, on='City')
tongCN

,City,Total,lat,lon
0,Hà Nội,85584.3555,21.029450,105.854444
1,TP HCM,161644.7490,10.762622,106.660172
2,Đà Nẵng,75737.6445,16.047079,108.206230


**Thử hiển thị bằng px.scatter_mapbox**

In [17]:
figMap = px.scatter_mapbox(tongCN, lat="lat", lon="lon",
                        hover_name="City", 
                        hover_data=["Total"],
                        size="Total",
                        zoom=4, height=400, width=250,
                        color=tongCN["City"],
                        color_discrete_sequence=["blue", "red", "yellow", ]
                       )
figMap.update_layout(mapbox_style="stamen-terrain")
# fig.update_layout(mapbox_style="open-street-map")

figMap.update_geos(fitbounds="locations", 
                showcountries=True,
                countrycolor="Blue")

figMap.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, showlegend=False)
figMap.show()

**Thử hiển thị bằng px.scatter_geo**

In [18]:
figMap2 = px.scatter_geo(tongCN, lat="lat", lon="lon",
                        hover_name="City", 
                        hover_data=["Total"],
                        size="Total",
                        height=500, width=350,
                        color=tongCN["City"],
                        projection="mercator",
                        fitbounds='locations',
                        text=tongCN["City"], 
                        title='BIỂU ĐỒ',
                       )
figMap2.update_geos( 
                showcountries=True,
                countrycolor="Blue")

figMap2.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, 
                      showlegend=False)
figMap2.show()

# 3. So sánh tổng giá trị đơn hàng 

**Theo nhóm các mặt hàng**

In [19]:
tongNH = df.groupby('Product line')['Total'].sum()
tongNH

Product line
Electronic accessories    54337.5315
Fashion accessories       54305.8950
Food and beverages        56144.8440
Health and beauty         49193.7390
Home and lifestyle        53861.9130
Sports and travel         55122.8265
Name: Total, dtype: float64

**Theo giới tính**

In [20]:
tongGT = df.groupby('Gender')['Total'].sum()
tongGT

Gender
Nam    155083.824
Nữ     167882.925
Name: Total, dtype: float64

**Viết hàm hiển thị so sánh doanh thu theo:**
- Product line của tất cả các thành phố hoặc từng thành phố
- Theo giới tính
- Theo Thành phố

In [21]:
def SoSanhDoanhThu(thongso, city = 'Tất cả'):
    if 'Tất cả'.lower() in city.lower():
        df_temp = df
    else:
        df_temp = df[df['City'] == city]
        
    df_counts = df_temp.groupby(thongso)['Total'].sum()
    
    fig = make_subplots(rows=1, cols=2, 
                        subplot_titles=("So Sánh Theo Tổng", "So Sánh Theo Tỷ Lệ"), 
                        column_widths=[0.6, 0.4],
                        specs=[[{"type": "xy"}, {"type": "domain"}]]
                       )
    goBar = go.Bar(x=df_counts.index, y=df_counts.values, 
                   width=0.3,
                   showlegend=False
                  )
    goPie = go.Pie(labels=df_counts.index, values=df_counts.values,
                   showlegend=True,
                   hole=.3,
                  )
    fig.add_trace(goBar, row=1, col=1)
    fig.add_trace(goPie, row=1, col=2)
    fig.update_layout(width=600, height=300, 
                      margin={"r":0,"t":0,"l":0,"b":0},
                      title=f'BIỂU ĐỒ SO SÁNH DOANH THU THEO: {thongso}')
    fig.show()

In [22]:
SoSanhDoanhThu('Product line', 'TP HCM')

In [24]:
SoSanhDoanhThu('Gender')

In [25]:
SoSanhDoanhThu('City')

**Thử kết hợp dropdown và bản đồ**

In [27]:
bCity = widgets.Select(description='Chọn Thành Phố',
                           options=['Tất Cả'] + list(tongCN['City']),
                           style = {'description_width': 'initial'}
                          )
mapWidget = go.FigureWidget(figMap2)
box1 = widgets.VBox([mapWidget, bCity])
display(box1)

    'data': [{'customdata': array([[85584.3555]]),
              'geo': 'geo',
 …

**Hiển thị thử theo các widgets**

In [31]:
def SoSanhDoanhThu1(thongso, city = 'Tất cả'):
    if 'Tất cả'.lower() in city.lower(): df_temp = df
    else: df_temp = df[df['City'] == city]
        
    df_counts = df_temp.groupby(thongso)['Total'].sum()    
    fig = make_subplots(rows=1, cols=2, 
                        subplot_titles=("So Sánh Theo Tổng", "So Sánh Theo Tỷ Lệ"), 
                        column_widths=[0.6, 0.4],
                        specs=[[{"type": "xy"}, {"type": "domain"}]]
                       )
    goBar = go.Bar(x=df_counts.index, y=df_counts.values, 
                   width=0.3,
                   showlegend=False
                  )
    goPie = go.Pie(labels=df_counts.index, values=df_counts.values,
                   showlegend=True,
                   hole=.3,
                  )
    fig.add_trace(goBar, row=1, col=1)
    fig.add_trace(goPie, row=1, col=2)
    fig.update_layout(width=600, height=300, 
                      margin={"r":0,"t":50,"l":50,"b":0},
                      title=f'BIỂU ĐỒ SO SÁNH DOANH THU THEO: {thongso}')
    return fig
fig1 = SoSanhDoanhThu1('Product line', 'TP HCM')
fig2 = SoSanhDoanhThu1('Gender', 'TP HCM')

fig1W = go.FigureWidget(fig1)
fig2W = go.FigureWidget(fig2)

box2 = widgets.VBox([fig1W, fig2W])
display(box2)

    'data': [{'showlegend': False,
              'type': 'bar',
              'u…

**Kết hợp các biểu đồ lại thành dashboard**

In [33]:
wOutput = widgets.Output(layout={'border': '0px solid black'})

labelLayout = {'border': '1px solid black', 'background':'green',
               'justify_content':'center', 'display':'flex'}
wLabel = widgets.Label(value='THÀNH PHỐ: TẤT CẢ', layout=labelLayout)

box2 = widgets.VBox([wLabel, wOutput])
box = widgets.HBox([box1, box2])
display(box)


def ThucHien():
    fig1 = SoSanhDoanhThu1('Product line')
    fig2 = SoSanhDoanhThu1('Gender')
    with wOutput:
        wOutput.clear_output()
        fig1.show()
        fig2.show()
ThucHien()

def ThucHien2(city):
    if city == 'Tất Cả':
        wLabel.value = 'THÀNH PHỐ: TẤT CẢ'
    else:
        wLabel.value = 'THÀNH PHỐ: ' + city
    fig1 = SoSanhDoanhThu1('Product line', city)
    fig2 = SoSanhDoanhThu1('Gender', city)
    with wOutput:
        wOutput.clear_output()
        fig1.show()
        fig2.show()
a = widgets.interactive(ThucHien2, city=bCity)

    'data': [{'customdata': array([[85584.3555]]),
              …